In [1]:
# Author: Aditya Ranganath
# Subject: Exploring optical flow and predicting frames based on Taylor expansion 

import cv2
# import torch
import random as rand
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image

In [2]:
def image_generation(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, minNorm, maxNorm, weight):
    cap = cv2.VideoCapture('./circle_noise1.avi')

    # We know the frame rate for the video is 10 fps
    deltat = 1/10

    # Compute the flow of two images and the compute then flow of flow with two flows
    # For that, extract one image from flow
    # then extract the next images in the loop

    _, prev= cap.read()

    Iot = prev

    hsv_mask = np.zeros_like(prev)

    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    flowl = []
    for i in range(2):    
        # we need only only flow sample and one flow of flow sample
        _, next = cap.read()

        hsv_mask = np.zeros_like(next)

        # Calculate the flow for a Black and white render
        next = cv2.cvtColor(next, cv2.COLOR_BGR2GRAY)

        #Compute the flow for these two images
        flow = cv2.calcOpticalFlowFarneback(prev, next, None, levels=levelsl, winsize=winsize1, iterations=iterations1, poly_n=poly_n1, poly_sigma=poly_sigma1, flags=flags1, pyr_scale=pyr_scale1)

        # Compute magnite and angle of 2D vector
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Set value as per the normalized magnitude of optical flow
        hsv_mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

        # Set image hue value according to the angle of optical flow
        hsv_mask[..., 0] = ang * 180 / np.pi / 2

        # Convert to rgb
        rgb_representation = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

        # Append to flow list
        flowl.append(rgb_representation)

        prev = next

        hsv_mask = np.zeros_like(prev)

    # Now compute the initial flow of flow
    fflow = cv2.calcOpticalFlowFarneback(cv2.cvtColor(flowl[0], cv2.COLOR_BGR2GRAY), cv2.cvtColor(flowl[1], cv2.COLOR_BGR2GRAY), f2, levels=levels2, winsize=winsize2, iterations=iterations2, poly_n=poly_n2,poly_sigma=poly_sigma2, flags=flags2, pyr_scale=pyr_scale2)
    
    # Compute magnite and angle of 2D vector
    mag, ang = cv2.cartToPolar(fflow[..., 0], fflow[..., 1])

    #Set the mask to zero again
    hsv_mask = np.zeros_like(flowl[0])

    # Set value as per the normalized magnitude of optical flow
    hsv_mask[..., 2] = cv2.normalize(mag, None, minNorm, maxNorm, cv2.NORM_MINMAX)

    # Now that we have the flow and flow of flows, lets calculate the Image!
    # The formulation for the image is given by
    # I(t + \Delta t) = I(t) + \Delta t * flow[0] + \frac{1}{2} \Delta t^2 fflow[0]

    # Convert to rgb
    fflow = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)

    # Computing the next frame
    Idt = lambda dt: Iot + flowl[1]*deltat + 0.5*fflow*deltat**2

    # Save to a list and return
    Idtl = []

    # Release and reset
    cap.release()

    # Read the file again to reset
    cap = cv2.VideoCapture('./circle_noise1.avi')

    #The new sampling rate
    newdeltat = 1/20

    i = 0

    for j in range(200):
        try:
            _, Iot = cap.read()
            Idtl.append(Iot)
            for i in range(2):
                
                weights = np.uint8(weight)
                Inext = Idt((i+1)*newdeltat)

                # weighted sum between frames
                mid_frame = cv2.addWeighted(np.uint8(Iot),np.uint8(weights),np.uint8(Inext),1-weight,0)

                Idtl.append(mid_frame)
                Idtl.append(np.uint8(Inext))
                i=i+1
                Iot= mid_frame
        except:
            break
    
    return Idtl

In [3]:

def gen(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, minNorm, maxNorm, weight):

    z = image_generation(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, minNorm, maxNorm, weight)
    print("Flow : {}, Levels: {}, Winsize: {}, Iterations: {}, Poly_n: {}, Poly_sigma: {}, Flags: {}, Pyr_Scale: {}, Flow2: {}, Levels 2: {}, Winsize 2: {}, Iterations 2 : {}, Poly_n 2: {}, Poly_Sigma 2: {}, Flags 2: {}, Pyr_Scale2: {}, MinNorm: {}, MaxNorm: {}, Weight: {}".format(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, minNorm, maxNorm, weight))
    return z

In [4]:
def count_original():
    cap = cv2.VideoCapture("./circle_noise1.avi")
    
    count = 0
    
    while (1):
        
        _, frame = cap.read()
        
        if _ == False:
            break
        count += 1
    
    print(count)
                           

In [5]:
def count_restored(i):
    cap = cv2.VideoCapture("./circle_noise_restored_%i.avi" % i)
    
    count = 0
    
    while (1):
        
        _, frame = cap.read()
        
        if _ == False:
            break
        count += 1
    
    print(count)

In [6]:
def count_normal_restored(i):
    cap = cv2.VideoCapture("./circle_noise_restored_normal_%d.avi" % i)
    
    count = 0
    
    while (1):
        
        _, frame = cap.read()
        
        if _ == False:
            break
        count += 1
    
    print(count)
    

In [7]:
def write(Idtl, i):
    fourcc = cv2.VideoWriter_fourcc(*'MP42')
    FPS = 40
    width, height = 64,64
    import numpy as np
    video = cv2.VideoWriter('./circle_noise_restored_%d.avi' % i, fourcc, float(FPS), (width, height))

    for frame in Idtl:
        try:
            video.write(frame)
        except:
            set_trace()
    video.release()

In [8]:
def write_normal(Idtl, i):
    fourcc = cv2.VideoWriter_fourcc(*'MP42')
    FPS = 40
    width, height = 64,64
    import numpy as np
    video = cv2.VideoWriter('./circle_noise_restored_normal_%d.avi' % i, fourcc, float(FPS), (width, height))

    for frame in Idtl:
        try:
            video.write(frame)
        except:
            set_trace()
    video.release()

In [9]:

def main():
    

    for i in range(10):


        random = randint(1, 5)

        f1 =  None


        levelsl = 6


        winsize1 = 3

        iterations1 = 4

        poly_n1 = 3

        poly_sigma1 = 1.2

        flags1 = cv2.OPTFLOW_FARNEBACK_GAUSSIAN

        pyr_scale1 = 0.5

        f2 = None

        levels2 = 3

        winsize2 = 3

        iterations2 = 7
        
        poly_n2 = 5

        poly_sigma2 = 1.2

        flags2 = cv2.OPTFLOW_FARNEBACK_GAUSSIAN

        pyr_scale2 =  0.5

        minNorm = randint(1, 128)

        maxNorm = randint(128, 255)

        weight =  round(rand.uniform(00.01, 00.97), 2)

        x = gen(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, minNorm, maxNorm, weight)
        y = gen(f1, levelsl, winsize1, iterations1, poly_n1, poly_sigma1, flags1, pyr_scale1, f2, levels2, winsize2, iterations2, poly_n2, poly_sigma2, flags2, pyr_scale2, 0, 255, weight)

        write(x, i+1)
        write_normal(y, i+1)
        count_original()
        count_restored(i+1)
        count_normal_restored(i+1)
        

                      

In [10]:
main()

Flow : None, Levels: 6, Winsize: 3, Iterations: 4, Poly_n: 3, Poly_sigma: 1.2, Flags: 256, Pyr_Scale: 0.5, Flow2: None, Levels 2: 3, Winsize 2: 3, Iterations 2 : 7, Poly_n 2: 5, Poly_Sigma 2: 1.2, Flags 2: 256, Pyr_Scale2: 0.5, MinNorm: 50, MaxNorm: 173, Weight: 0.57
Flow : None, Levels: 6, Winsize: 3, Iterations: 4, Poly_n: 3, Poly_sigma: 1.2, Flags: 256, Pyr_Scale: 0.5, Flow2: None, Levels 2: 3, Winsize 2: 3, Iterations 2 : 7, Poly_n 2: 5, Poly_Sigma 2: 1.2, Flags 2: 256, Pyr_Scale2: 0.5, MinNorm: 0, MaxNorm: 255, Weight: 0.57
12
60
60
Flow : None, Levels: 6, Winsize: 3, Iterations: 4, Poly_n: 3, Poly_sigma: 1.2, Flags: 256, Pyr_Scale: 0.5, Flow2: None, Levels 2: 3, Winsize 2: 3, Iterations 2 : 7, Poly_n 2: 5, Poly_Sigma 2: 1.2, Flags 2: 256, Pyr_Scale2: 0.5, MinNorm: 53, MaxNorm: 129, Weight: 0.5
Flow : None, Levels: 6, Winsize: 3, Iterations: 4, Poly_n: 3, Poly_sigma: 1.2, Flags: 256, Pyr_Scale: 0.5, Flow2: None, Levels 2: 3, Winsize 2: 3, Iterations 2 : 7, Poly_n 2: 5, Poly_Sigma 

In [11]:
# cv2.calcOpticalFlowFarneback??

In [12]:
# cv2.imshow('flowoflfow',fflow)
# plt.imshow(fflow)
# plt.imshow(flowl[1])